In [ ]:
# Libraries and Dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
# from ANOTHER MACHINE LEARNING LIBRARY import SOMETHING ELSE

# APIs
# Use Alpaca for BTC data

# Inputs

In [ ]:
# Read in Data Files

# Data Processing

# Machine Learning Analysis

# Save each of your alternative models as an HDF5 file

# Output
## Connect to an External Exchange (Demo Account)
or
## Send an SMS (Backup)

# Nice to have: Sentiment Analysis APIs